In [68]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [72]:
df = pd.read_csv('tennis_data.csv')

C:\Users\pariz\AppData\Local\Temp\ipykernel_24384\3316052404.py:1: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('tennis_data.csv')


In [60]:
# průměr kurzů
df['pl1_avg_bookmaker_odds'] = df[['B365W', 'PSW', 'MaxW', 'AvgW', 'LBW', 'SJW', 'UBW']].mean(axis=1)
df['pl2_avg_bookmaker_odds'] = df[['B365L', 'PSL', 'MaxL', 'AvgL', 'LBL', 'SJL', 'UBL']].mean(axis=1)

# drop zbytečných sloupců
df = df.drop(['ATP', 'Location', 'Tournament', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets', 'B365W', 'B365L', 'PSW', 'PSL', 'MaxW', 'MaxL', 'AvgW', 'AvgL', 'EXW', 'EXL', 'LBW', 'LBL', 'SJW', 'SJL', 'UBW', 'UBL', 'pl1_flag', 'pl2_flag'], axis=1)

# ponechání pouze dokončených zápasů
df = df[df['Comment'] == 'Completed']
df = df.drop(['Comment'], axis=1)
df = df.dropna()

# kódování textových sloupců
col = [['series_code','Series'],['court_code','Court'],['round_code','Round'],['surface_code','Surface'],['best_of','Best of'],['pl1_hand_code','pl1_hand'],['pl2_hand_code','pl2_hand']]
for i in range(len(col)):
    df[col[i][0]] = df[col[i][1]].astype('category').cat.codes
df = df.drop(['Series', 'Court', 'Surface', 'Round', 'Best of', 'pl1_hand', 'pl2_hand'], axis=1)

# úprava formátu date
df['date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df = df.drop(['Date'], axis=1) 

# odstranění odlehlých hodnot ve výšce a váze obou hráčů
df = df.drop(df[df['pl1_weight'] < 50].index)
df = df.drop(df[df['pl1_height'] < 100].index)
df = df.drop(df[df['pl1_height'] > 220].index)
df = df.drop(df[df['pl2_weight'] < 50].index)
df = df.drop(df[df['pl2_height'] < 100].index)
df = df.drop(df[df['pl2_height'] > 220].index)

# odstranění odlehlých hodnot ve vypsaných kurzech bookmakerů
df = df.drop(df[df['pl1_avg_bookmaker_odds'] > 20].index)
df = df.drop(df[df['pl2_avg_bookmaker_odds'] > 20].index)
df = df.drop(df[(df['pl1_avg_bookmaker_odds'] > 5) & (df['pl2_avg_bookmaker_odds'] > 5)].index)

# tvorba win_rate a total_games a merge s originálním df
wins = df['Winner'].value_counts().reset_index()
wins.columns = ['Player', 'Wins']
losses = df['Loser'].value_counts().reset_index()
losses.columns = ['Player', 'Losses']

player_stats = pd.merge(wins, losses, on='Player', how='outer').fillna(0)

player_stats['Win_Rate'] = player_stats['Wins'] / (player_stats['Wins'] + player_stats['Losses'])

player_stats['total_games'] = player_stats['Wins'] + player_stats['Losses']

df = pd.merge(df, player_stats[['Player', 'total_games', 'Win_Rate']], left_on='Winner', right_on='Player', how='left')
df = pd.merge(df, player_stats[['Player', 'total_games', 'Win_Rate']], left_on='Loser', right_on='Player', how='left')

df = df.drop(['Player_x', 'Player_y'], axis=1)
df = df.dropna()

# zachování konvence jmen
df.rename(columns={'Win_Rate_x':'pl1_win_rate', 'Win_Rate_y':'pl2_win_rate', 'total_games_x':'pl1_total_games', 'total_games_y':'pl2_total_games'}, inplace=True)
df.rename(columns={'Winner':'pl1', 'Loser':'pl2', 'WRank':'pl1_rank', 'LRank':'pl2_rank', 'WPts':'pl1_pts', 'LPts':'pl2_pts'}, inplace=True)

# tvorba swrate
surface_win_counts = df.groupby(['pl1', 'surface_code']).size().reset_index(name='wins')
surface_lose_counts = df.groupby(['pl2', 'surface_code']).size().reset_index(name='losses')

merged_counts = pd.merge(surface_win_counts, surface_lose_counts, left_on=['pl1', 'surface_code'], right_on=['pl2', 'surface_code'], how='outer')
merged_counts['surface_win_rate'] = merged_counts['wins'] / (merged_counts['losses'] + merged_counts['wins'])

df = pd.merge(df, merged_counts[['pl1', 'surface_code', 'surface_win_rate']], left_on=['pl1', 'surface_code'], right_on=['pl1', 'surface_code'], how='left')
df = pd.merge(df, merged_counts[['pl2', 'surface_code', 'surface_win_rate']], left_on=['pl2', 'surface_code'], right_on=['pl2', 'surface_code'], how='left')
df = df.dropna()

# tvorba rozdílů mezi hráči
df.rename(columns={'surface_win_rate_x':'pl1_swrate', 'surface_win_rate_y':'pl2_swrate'}, inplace=True)
df['rank_diff'] = df.apply(lambda row: max(row['pl1_rank'], row['pl2_rank']) - min(row['pl1_rank'], row['pl2_rank']), axis=1)
df['pts_diff'] = df.apply(lambda row: max(row['pl1_pts'], row['pl2_pts']) - min(row['pl1_pts'], row['pl2_pts']), axis=1)
df['wrate_diff'] = df.apply(lambda row: max(row['pl1_win_rate'], row['pl2_win_rate']) - min(row['pl1_win_rate'], row['pl2_win_rate']), axis=1)
df['swrate_diff'] = df.apply(lambda row: max(row['pl1_swrate'], row['pl2_swrate']) - min(row['pl1_swrate'], row['pl2_swrate']), axis=1)

# vytvoření závislé proměnné
df['result'] = 1

# vytvoření id sloupce
df['id'] = range(1, len(df) + 1)

# balancování závislé proměnné
even_df = df[df['id'] % 2 == 0]
odd_df = df[df['id'] % 2 != 0]

values_to_swap = [['pl1','pl2'],["pl1_hand_code","pl2_hand_code"],["pl1_rank","pl2_rank"],["pl1_pts","pl2_pts"],["pl1_year_pro","pl2_year_pro"],["pl1_weight","pl2_weight"],["pl1_height","pl2_height"],['pl1_avg_bookmaker_odds','pl2_avg_bookmaker_odds'],['pl1_total_games','pl2_total_games'],['pl1_win_rate','pl2_win_rate'],['pl1_swrate','pl2_swrate']]
for i in range(len(values_to_swap)):
    odd_df[values_to_swap[i][0]], odd_df[values_to_swap[i][1]] = odd_df[values_to_swap[i][1]], odd_df[values_to_swap[i][0]]

odd_df['result'] = 0

# vytvoření finálního df spojením lichých a sudých df
final_df = pd.concat([even_df, odd_df], ignore_index=True)
final_df = final_df.sort_values(by='id')

C:\Users\pariz\AppData\Local\Temp\ipykernel_24384\969605949.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odd_df[values_to_swap[i][0]], odd_df[values_to_swap[i][1]] = odd_df[values_to_swap[i][1]], odd_df[values_to_swap[i][0]]
C:\Users\pariz\AppData\Local\Temp\ipykernel_24384\969605949.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odd_df['result'] = 0


In [62]:
final_df.to_csv('output.csv', index=False)

In [64]:
# Výpočet prediktivní přesnosti kurzů od bookmakerů
test_df = pd.read_csv('output.csv')
test_df = test_df[test_df["date"] >= "2019-01-01"]

test_df['bm_match'] = ''

for index, row in test_df.iterrows():
    if (row['pl1_avg_bookmaker_odds'] < row['pl2_avg_bookmaker_odds']) and row['result'] == 1:
        test_df.at[index, 'bm_match'] = 1
    elif (row['pl1_avg_bookmaker_odds'] > row['pl2_avg_bookmaker_odds']) and row['result'] == 0:
        test_df.at[index, 'bm_match'] = 1
    else:
        test_df.at[index, 'bm_match'] = 0

wins = (test_df['bm_match'] == 1).sum()
losses = (test_df['bm_match'] == 0).sum()

accuracy = wins / (wins + losses)
print('Bookmaker Accuracy:')
print('===================')
print('Wins:    ', wins)
print('Losses:  ', losses)
print('Accuracy:', round(accuracy*100,2), '%')
print('===================')



Bookmaker Accuracy:
Wins:     4196
Losses:   2088
Accuracy: 66.77 %
